# データセットの作成（Create a dataset）

ときどき，独自のデータを扱う場合にデータセットを作成する必要が生じます．
Hugging Face Datasets でデータセットを作成することは，ライブラリが提供する全ての利点をデータセットに与えることができます．
例えば，高速な読み込みと処理，膨大なデータセットのストリーミング，メモリマッピングなどです．
Hugging Face Datasetsを使うことで，少ないコーディングで簡単に迅速にデータセットの作成ができ，モデルの学習の開始にかかる時間を短縮できます．
多くの場合，データファイルをHubのリポジトリにドラッグアンドドロップするように簡単です．

このチュートリアルでは，全てのタイプのデータセットを，Hugging Face Datasetsの少ないコーディングで作成する方法を紹介します．
- 画像やオーディオのデータセットを素早く作成するためのフォルダベースのビルダ
- ローカルファイルからデータセットを作成する`from_`メソッド

## フォルダベースのビルダ（Folder-based builder）

2種類のフォルダベースのビルダがあります．
一つは`ImageFolder`，もう一つは`AudioFolder`です．
これらは，少ないコーディング（low-code method）で数千の画像やオーディオファイルを含むデータセットを作成するために設計されています．
画像や音声のモデルを作る際，より大きいデータセットにスケーリングする前の素早いプロトタイピングをするのに優れています．
フォルダーベースビルダは，データを受け取ると自動的にデータセットのフィーチャ（features），スプリット（splits），そしてラベル（labels）を生成します:

- `ImageFolder`は`Image`特徴を持ち，画像ファイルをデコードするのに用います．
多くの画像拡張形式（jpgやpng，その他）がサポートされています．
[このリスト](https://github.com/huggingface/datasets/blob/b5672a956d5de864e6f5550e493527d962d6ae55/src/datasets/packaged_modules/imagefolder/imagefolder.py#L39)でサポートされる全ての画像形式が確認できます．
- `AudioFolder`は`Audio`特徴を持ち，音声ファイルをデコードするのに用います．
音声ファイルの形式は，wavやmp3などがサポートされています．
[このリスト](https://github.com/huggingface/datasets/blob/b5672a956d5de864e6f5550e493527d962d6ae55/src/datasets/packaged_modules/audiofolder/audiofolder.py#L39)でサポートされる全ての音声形式が確認できます．

データセットのスプリットはリポジトリの構造から生成されます，
また，ラベル名はディレクトリ名から自動的に推測されます．

例えば，もしあなたの画像データが次のように保存されているとします（音声の場合も同様です）:
```
pokemon/train/grass/bulbasaur.png
pokemon/train/fire/charmander.png
pokemon/train/water/squirtle.png

pokemon/test/grass/ivysaur.png
pokemon/test/fire/charmeleon.png
pokemon/test/water/wartortle.png
```
この場合，フォルダベースのビルダは次のように解釈します:
- `train`や`test`はスプリット名
- `grass`, `fire`, `water`はラベル名
- `bulbasaur.png`, `charmander.png`, `squirtle.png`は画像ファイル

`load_dataset()`の`imagefolder`を指定することで，画像データセットを作成できます．

In [1]:
from datasets import load_dataset
dataset = load_dataset("imagefolder", data_dir="./example_data/image")

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

オーディオデータセットも同様に，`load_dataset()`で`audiofolder`を指定することで作成できます．

In [2]:
from datasets import load_dataset
dataset = load_dataset("audiofolder", data_dir="./example_data/audio")

EmptyDatasetError: The directory at /mnt/data1/home/fujie/work/2024/huggingface_tutorial/datasets/tutorials/example_data/audio doesn't contain any data files

データセットに関する追加情報（テキストキャプションや，トランスクリプションなど）は，データセットのフォルダの中の`metadata.csv`ファイルに含めることができます．
メタデータファイルは，メタデータに関連する画像やオーディオファイルへのリンクを含む`file_name`列を持つ必要があります:
```
file_name, text
bulbasaur.png, There is a plant seed on its back right from the day this Pokémon is born.
charmander.png, It has a preference for hot things.
squirtle.png, When it retracts its long neck into its shell, it squirts out water with vigorous force.
```
フォルダベースビルダに関する詳細は，[ImageFolder](https://huggingface.co/docs/datasets/image_dataset#imagefolder)ガイドや[AudioFolder](https://huggingface.co/docs/datasets/audio_dataset#audiofolder)ガイドを参照してください．

## ローカルファイルからの作成（From local files）

パスとデータファイルを指定することでローカルファイルからデータセットを作ることもできます．
`from_`メソッドを用いてデータセットを作る方法は2通りあります:
- `from_generator()`メソッドはジェネレータを用いてデータセットを作る，メモリ効率のよい方法です．
この方法は，データセットがディスクから徐々に読み込まれてメモリにマッピングされるため，全てをメモリに置くことが適さない大きなデータを扱うときに有用です．

In [14]:
from datasets import Dataset
def gen():
    yield {"pokemon": "bulbasur", "type": "grass"}
    yield {"pokemon": "squirtle", "type": "water"}

ds = Dataset.from_generator(gen)
ds[0]

Generating train split: 0 examples [00:00, ? examples/s]

{'pokemon': 'bulbasur', 'type': 'grass'}

ジェネレータベースの`IterableDataset`は`for`ループを使ってデータを読み込むことができます:

In [15]:
from datasets import IterableDataset
ds = IterableDataset.from_generator(gen)
for example in ds:
    print(example)

{'pokemon': 'bulbasur', 'type': 'grass'}
{'pokemon': 'squirtle', 'type': 'water'}


`from_dict()`メソッドは，辞書からデータを作成する方法です:

In [16]:
from datasets import Dataset
ds = Dataset.from_dict({"pokemon": ["bulbasur", "squirtle"], "type": ["grass", "water"]})
ds[0]

{'pokemon': 'bulbasur', 'type': 'grass'}

画像やオーディオのデータセットを作成するには，`cast_column()`メソッドを`from_dict()`につなげて，列や特徴を指定します．
例えばオーディオデータセットを作成するには以下のようにします:

In [19]:
from datasets import Dataset, Audio

audio_dataset = Dataset.from_dict(
    {"audio": ["example_data/audio/pokemon/train/fire/charmander.wav",
                "example_data/audio/pokemon/train/fire/charmeleon.wav",
                "example_data/audio/pokemon/train/fire/charizard.wav",
                "example_data/audio/pokemon/train/water/squirtle.wav",
                "example_data/audio/pokemon/train/water/wartortle.wav",
                "example_data/audio/pokemon/train/water/blastoise.wav"],
      "label": ["fire", "fire", "fire", "water", "water", "water"]}).cast_column("audio", Audio())

## 次のステップ（Next steps）

このチュートリアルでは触れませんでしたが，読み込みスクリプト（loading script）を使用してデータセットを作成することもできます．
読み込みスクリプトはより手動的で，多くのコードを利用する方法ですが，データセットの生成方法について完全にフレキシブルな方法です．
この方法により，追加情報をデータセットに与えることができます．例えば，複数のコンフィギュレーションをデータに与えたり，データをストリーム対応にしたりすることができます．

読み込みスクリプトの書き方の詳細については，[image loading script](https://huggingface.co/docs/datasets/main/en/image_dataset#loading-script)，[audio loading script](https://huggingface.co/docs/datasets/main/en/audio_dataset)，[text loading script](https://huggingface.co/docs/datasets/main/en/dataset_script)ガイドを参照してください．

データセットの作成方法がわかったので，Hubにデータを共有してあなたの仕事からコミュニティが恩恵をえることを検討してください．
共有方法については次の章で説明します．